<a href="https://colab.research.google.com/github/41371130H/PL-Repo/blob/main/HW1_%E6%97%A5%E5%B8%B8%E6%94%AF%E5%87%BA%E9%80%9F%E7%AE%97%E8%88%87%E5%88%86%E6%94%A4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 日常支出速算與分攤（作業一）
*   目標：從 Sheet 讀「消費紀錄」→ 計總額/分類小計/AA 分攤 → 寫回 Sheet Summary 分頁。
*   AI 點子（可選）：請模型總結本週花錢習慣與建議（例如「外食過多」）。
*   Sheet 欄位：date, category, item, amount, payer



In [14]:
from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)

import pandas as pd
# read data and put it in a dataframe
# 在 google 工作表載入 gsheets
gsheets = gc.open_by_url('https://docs.google.com/spreadsheets/d/1KhaWfYSh2fGRmwC8SkZG8GdbTshYql4QX-hknk0ON84/edit?usp=sharing')
worksheet = gsheets.worksheet('工作表1')

# 從 gsheets 的 All-whiteboard-device 載入 sheets
sheets = gsheets.worksheet('工作表1').get_all_values()
# 將 sheets1 資料載入 pd 的 DataFrame 進行分析
df = pd.DataFrame(sheets[1:], columns=sheets[0])
# 取得最前面的5筆資料
df.head()

# ==============================
# 功能選單
# ==============================
action = input("請選擇動作（1: 新增資料, 2: 刪除資料, 3: 統整資料）: ")

if action == "1":

  import datetime
  import pytz

  # 設定台灣時區
  tz = pytz.timezone("Asia/Taipei")
  now = datetime.datetime.now(tz)
  date_str = now.strftime("%Y-%m-%d")  # 例如：2025-09-11
  time_str = now.strftime("%H:%M")     # 例如：10:25

  # 讓使用者輸入資料
  categorynumber = input("請輸入項目（1: 早餐, 2: 午餐, 3: 晚餐, 4: 飲料, 5: 交通, 6: 就學用品, 7: 日常用品）: ")
  item = input("請輸入品項: ")
  amount = float(input("請輸入金額: "))
  aa = input("是否要AA分攤（yes or no）: ")
  if aa == "yes":
    people = int(input("請輸入分攤人數: "))
    amount = amount // people
    print(f"這筆支出實付 {amount} 元")
  else:
    amount = amount

  if categorynumber == "1":
    category = "早餐"
  elif categorynumber == "2":
    category = "午餐"
  elif categorynumber == "3":
    category = "晚餐"
  elif categorynumber == "4":
    category = "飲料"
  elif categorynumber == "5":
    category = "交通"
  elif categorynumber == "6":
    category = "就學用品"
  elif categorynumber == "7":
    category = "日常用品"
  else:
    category = "其他"



  # 創建一個新的 DataFrame 來代表你要新增的資料
  new_data = pd.DataFrame([{
      '日期': date_str,
      '時間': time_str,
      '項目': category,
      '品項': item,
      '金額': amount }])

  # # 使用 concat() 將新資料合併到舊的 df 中
  # df = pd.concat([df, new_data], ignore_index=True)
  # # 將 DataFrame 轉換成列表的列表 (list of lists)，這是 gspread 支援的資料格式
  # data_to_write = df.values.tolist()

  data_to_write = new_data.values.tolist()

  # 第二步：將資料寫入工作表
  worksheet.append_rows(values=data_to_write, value_input_option='USER_ENTERED')
  print("資料已成功寫入 Google 工作表！")


elif action == "2":
    # 🔹 讀取目前資料
    all_data = worksheet.get_all_values()
    df = pd.DataFrame(all_data[1:], columns=all_data[0])  # 去掉標題列

    # 顯示現有資料給使用者看
    print("\n目前資料：")
    print(df)

    # 輸入要刪除的列號（對應 Google Sheet）
    row_to_delete = int(input("\n請輸入要刪除的列號: "))

    # 執行刪除
    worksheet.delete_rows(row_to_delete+2)
    print(f"第 {row_to_delete} 列號已成功刪除！")


elif action == "3":
    # 🔹 讀取資料
    all_data = worksheet.get_all_values()
    df = pd.DataFrame(all_data[1:], columns=all_data[0])  # 去掉標題列

    # 確保金額欄位是數字
    df["金額"] = pd.to_numeric(df["金額"], errors="coerce")

    # 計算總金額
    total_amount = df["金額"].sum()

    # 計算每個品項的總額，並排序
    category_summary = df.groupby("項目")["金額"].sum().sort_values(ascending=False)

    print("\n統整結果：")
    print(f"所有金額總和：{total_amount} 元\n")
    print("各項目金額排行：")
    print(category_summary)

else:
    print("輸入錯誤，請輸入 1、2 或 3。")

請選擇動作（1: 新增資料, 2: 刪除資料, 3: 統整資料）: 3

統整結果：
所有金額總和：1020 元

各項目金額排行：
項目
就學用品    800
飲料      100
午餐       60
晚餐       50
日常用品     10
Name: 金額, dtype: int64
